In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
import os
import random
path = os.path.join(os.getcwd(), "exp")
if not os.path.exists(path):
  os.mkdir(path)

In [ ]:
from numpy.lib import utils
densenet_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

alexnet_transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
transform = transforms.Compose([
    transforms.ToTensor(),
])

tranining_size = 10000
testing_size = 10000
label_noise = 0.2
batch_size = 100
torch.random.manual_seed(6699)
noise_data = datasets.FashionMNIST(root = './data', train = True, download = True, transform= transform)
original_data = datasets.FashionMNIST(root = './data', train = True, download = True, transform= transform)
test_data = datasets.FashionMNIST(root = './data', train = False, download = True, transform= transform)
noise_label = noise_data.targets
change_label = np.random.uniform(0, 1, len(noise_label)) < label_noise
for idx in range(len(noise_label)):
  if change_label[idx]:
    new_label = random.choice([i for i in range(10) if i != noise_label[idx]])
    noise_label[idx] = new_label
noise_data.targets = noise_label
noise_data, validate_data = torch.utils.data.random_split(noise_data, [tranining_size, len(noise_data) - tranining_size])
test_data, _ = torch.utils.data.random_split(test_data, [testing_size, len(test_data) - testing_size])
#random_index = np.random.sample(range(tranining_size), int(tranining_size*label_noise))
#random_new = np.random.shuffle(train_targets[random_index])
#train_targets[random_index] = random_new
#train_data.targets = train_targets.tolist()
#noise_data = torch.clone(train_data)
#noise_data.targets = random_new.tolist()
#noise_data.data = train_data.data[random_index]
train_loader = DataLoader(noise_data, batch_size = batch_size, shuffle = True, num_workers = 1, pin_memory = True)
validate_loader = DataLoader(validate_data, batch_size = batch_size, shuffle = False, num_workers = 1, pin_memory = True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False, num_workers = 1, pin_memory = True)
dataloaders = {'train': train_loader, 'test': test_loader, 'valid': validate_data}

In [ ]:
class DenseNet(nn.Module):
    def __init__(self, channel_first = True, f1 = 128, f2 = 32, channel_num = 1, layers = [6, 12, 24, 16]):
        super().__init__()
        self.channel_first = channel_first
        self.f1 = f1
        self.f2 = f2
        self.channel_num = channel_num
        self.layers = layers
    
    def get_Conv_layer(self, in_channels):
        conv_layer = nn.Sequential(
            nn.Conv2d(in_channels, 64, (7, 7), (2, 2), (3, 3)),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0, inplace = True),
            nn.MaxPool2d(3, 2, 1)
        )
        return conv_layer

    def get_Trans_layer(self, in_channels):
        trans_layer = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.LeakyReLU(0, inplace = True),
            nn.Conv2d(in_channels, in_channels // 2, (1, 1), (1, 1)),
            nn.AvgPool2d(2, 2)
        )
        return trans_layer
    
    def get_Dense_layer(self, in_channels):
        dense_layer = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.LeakyReLU(0, inplace = True),
            nn.Conv2d(in_channels, self.f1, (1, 1), (1, 1)),
            nn.BatchNorm2d(self.f1),
            nn.LeakyReLU(0, inplace = True),
            nn.Conv2d(self.f1, self.f2, (3, 3), (1, 1), padding = 'same')
        )
        return dense_layer
    
    def get_Output_layer(self, in_channels):
        output_layer = nn.Linear(in_channels, 1000)
        return output_layer

    def forward(self, Input):
        if not self.channel_first:
            Input = Input.transpose(1, 2)
            Input = Input.transpose(1, 3)
        self.channel_num = Input.shape[1]
        conv_layer = self.get_Conv_layer(Input.shape[1])
        Output = conv_layer(Input)
        for idx in range(len(self.layers)):
            layer_num = self.layers[idx]
            for _ in range(layer_num):
                dense_layer = self.get_Dense_layer(Output.shape[1])
                Output_new = dense_layer(Output)
                Output = torch.concat((Output, Output_new), dim = 1)
            if idx != len(self.layers) - 1:
                trans_layer = self.get_Trans_layer(Output.shape[1])
                Output = trans_layer(Output)
            else:
                Output = nn.BatchNorm2d(Output.shape[1])(Output)
                Output = torch.mean(Output, dim = (2, 3))
                output_layer = self.get_Output_layer(Output.shape[1])
                Output = output_layer(Output)
        return Output



In [ ]:
class AlexNet(nn.Module):
    def __init__(self, channel_first = True, f1 = 128, f2 = 32, channel_num = 1, layers = [6, 12, 24, 16]):
        super().__init__()
        self.pretrained_alexnet = models.alexnet(pretrained = False)
        self.convlayer = self.pretrained_alexnet.features[:12]
        self.maxlayer = nn.MaxPool2d(3, 2)
        self.avglayer = nn.AdaptiveAvgPool2d(output_size=(6, 6))
        self.fclayer = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.LeakyReLU(0, inplace = True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.LeakyReLU(0, inplace = True),
            nn.Linear(4096, 1000),
            nn.LeakyReLU(0, inplace = True),
            nn.Linear(1000, 10)
        )
        #self.choose_conv_layers()

    def choose_conv_layers(self):
      for name, param in self.convlayer.named_parameters():
        number = int(name.split('.')[0])
        if number != 10:
          param.requires_grad = False
        else:
          param.requires_grad = True    
            
            
    def forward(self, x):
        x = self.convlayer(x)
        x = self.maxlayer(x)
        x = self.avglayer(x)
        x = nn.Flatten()(x)
        x = self.fclayer(x)
        return x


In [ ]:
class sk_CNN(nn.Module):
  def __init__(self, filter_num):
    super(sk_CNN, self).__init__()
    self.convlayer1 = nn.Sequential(
        nn.Conv2d(1, filter_num, 3, 1),
        nn.BatchNorm2d(filter_num),
        nn.LeakyReLU(0.2, inplace = True)
    )
    self.convlayer2 = nn.Sequential(
        nn.Conv2d(filter_num, filter_num, 3, 1),
        nn.BatchNorm2d(filter_num),
        nn.LeakyReLU(0.2, inplace = True)
    )
    self.convlayer3 = nn.Sequential(
        nn.Conv2d(filter_num, filter_num, 3, 1),
        nn.BatchNorm2d(filter_num),
        nn.LeakyReLU(0.2, inplace = True)
    )
    self.fclayer = nn.Linear(484 * filter_num, 10)
  def forward(self, x):
    x = self.convlayer1(x)
    x = self.convlayer2(x)
    x = self.convlayer3(x)
    x = nn.Flatten()(x)
    x = self.fclayer(x)
    
    return x

In [ ]:
class Epoch_net(nn.Module):
  def __init__(self):
    super(Epoch_net, self).__init__()
    self.convlayer1 = nn.Sequential(
        nn.Conv2d(1, 64, 3, 1, padding = 'same'),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )
    self.convlayer2 = nn.Sequential(
        nn.Conv2d(64, 128, 3, 1, padding = 'same'),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.convlayer3 = nn.Sequential(
        nn.Conv2d(128, 256, 3, 1, padding = 'same'),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.convlayer4 = nn.Sequential(
        nn.Conv2d(256, 512, 3, 1, padding = 'same'),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.faclayer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.convlayer1(x)
    x = self.convlayer2(x)
    x = self.convlayer3(x)
    x = self.convlayer4(x)
    x = self.faclayer(x)

    return x

In [ ]:
class Width_net4(nn.Module):
  def __init__(self, width = 64):
    super(Width_net4, self).__init__()
    self.convlayer1 = nn.Sequential(
        nn.Conv2d(1, width, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width),
        nn.ReLU()
    )
    self.convlayer2 = nn.Sequential(
        nn.Conv2d(width, width*2, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width*2),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.convlayer3 = nn.Sequential(
        nn.Conv2d(width*2, width*4, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width*4),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.convlayer4 = nn.Sequential(
        nn.Conv2d(width*4, width*8, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width*8),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.faclayer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(width*72, 10)
    )
  def forward(self, x):
    x = self.convlayer1(x)
    x = self.convlayer2(x)
    x = self.convlayer3(x)
    x = self.convlayer4(x)
    x = self.faclayer(x)

    return x

In [ ]:
class Width_net2(nn.Module):
  def __init__(self, width = 64):
    super(Width_net2, self).__init__()
    self.convlayer1 = nn.Sequential(
        nn.Conv2d(1, width, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.convlayer2 = nn.Sequential(
        nn.Conv2d(width, width*2, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width*2),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.faclayer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(width*98, 10)
    )
  def forward(self, x):
    x = self.convlayer1(x)
    x = self.convlayer2(x)
    x = self.faclayer(x)

    return x

In [ ]:
class Width_net3(nn.Module):
  def __init__(self, width = 64):
    super(Width_net3, self).__init__()
    self.convlayer1 = nn.Sequential(
        nn.Conv2d(1, width, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.convlayer2 = nn.Sequential(
        nn.Conv2d(width, width*2, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width*2),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.convlayer3 = nn.Sequential(
        nn.Conv2d(width*2, width*4, 3, 1, padding = 'same'),
        nn.BatchNorm2d(width*4),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.faclayer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(width*36, 10)
    )
  def forward(self, x):
    x = self.convlayer1(x)
    x = self.convlayer2(x)
    x = self.convlayer3(x)
    x = self.faclayer(x)

    return x

In [ ]:
class Width_net1(nn.Module):
  def __init__(self, width = 64):
    super(Width_net1, self).__init__()
    self.convlayer1 = nn.Sequential(
        nn.Conv2d(1, width, 3, 1),
        nn.BatchNorm2d(width),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.faclayer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(width*169, 10)
    )
  def forward(self, x):
    x = self.convlayer1(x)
    x = self.faclayer(x)

    return x

In [ ]:
class Fully_Connected(nn.Module):
  def __init__(self, feature_size):
    super(Fully_Connected, self).__init__()
    self.fclayer1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, feature_size),
        nn.LeakyReLU(0, inplace = True)
    )
    self.fclayer2 = nn.Linear(feature_size, 10)

  def forward(self, x):
    x = self.fclayer1(x)
    x = self.fclayer2(x)
    return x

In [ ]:
class SGLD(optim.Optimizer):
  def __init__(self, params, lr=1e-2, precondition_decay_rate=0.95, num_pseudo_batches=1, num_burn_in_steps=3000, diagonal_bias=1e-8) -> None:
    if lr < 0.0:
      raise ValueError("Invalid learning rate: {}".format(lr))
    if num_burn_in_steps < 0:
      raise ValueError("Invalid num_burn_in_steps: {}".format(num_burn_in_steps))
    defaults = dict(
          lr=lr, precondition_decay_rate=precondition_decay_rate,
          num_pseudo_batches=num_pseudo_batches,
          num_burn_in_steps=num_burn_in_steps,
          diagonal_bias=1e-8,
          )
    super().__init__(params, defaults)

  
  def step(self, closure=None):
    loss = None
    if closure is not None:
      loss = closure()
    for group in self.param_groups:
      for parameter in group["params"]:
        if parameter.grad is None:
          continue
        state = self.state[parameter]
        lr = group["lr"]
        num_pseudo_batches = group["num_pseudo_batches"]
        precondition_decay_rate = group["precondition_decay_rate"]
        gradient = parameter.grad.data
        if len(state) == 0:
          state["iteration"] = 0
          state["momentum"] = torch.ones_like(parameter)
        state["iteration"] += 1
        momentum = state["momentum"]
        momentum.add_(
            (1.0 - precondition_decay_rate) * ((gradient ** 2) - momentum)
            )
        if state["iteration"] > group["num_burn_in_steps"]:
          sigma = 1. / torch.sqrt(torch.tensor(lr))
        else:
          sigma = torch.zeros_like(parameter)
        preconditioner = (
            1. / torch.sqrt(momentum + group["diagonal_bias"])
            )
        scaled_grad = (
            0.5 * preconditioner * gradient * num_pseudo_batches +
            lr * torch.normal(
                mean=torch.zeros_like(gradient),
                std=torch.ones_like(gradient)
                ) * sigma * torch.sqrt(preconditioner)
                )

        parameter.data.add_(-lr * scaled_grad)

    return loss

In [ ]:
class Classifier():
    def __init__(self, model, dataloaders, batch_size, use_cuda = True):
        self.dataloaders = dataloaders
        self.batch_size  = batch_size
        self.gpu = use_cuda
        if self.gpu and not torch.cuda.is_available():
          raise Exception("No GPU on this device!")
        self.model = model.to('cuda' if self.gpu else 'cpu')
        self.criterion = F.cross_entropy
        self.optimizer = SGLD(model.parameters(), lr = 0.01)
        lambda1 = lambda epoch: 1/(epoch + 1)**0.5
        lambda2 = lambda epoch: 1
        self.scheduler = torch.optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda = lambda1)

    def experiment_filter_num(self, epochs, change_lr = True):
      #best_val_err = 100
      best_train_err = 100
      best_test_err = 100
      best_train_loss = 0
      best_test_loss = 0
      epoch_test_err = []
      epoch_test_loss = []
      for epoch in range(epochs):
        print("Epoch: " +  str(epoch + 1))
        self.model.train()
        train_batch_num = len(self.dataloaders['train'])
        train_loss = 0
        train_error = 0
        for index, data in enumerate(self.dataloaders['train']):
          inputs, labels = data
          inputs = inputs.to('cuda' if self.gpu else 'cpu')
          labels = labels.to('cuda' if self.gpu else 'cpu')
          self.optimizer.zero_grad()
          outputs = self.model(inputs)
          loss = self.criterion(outputs, labels)
          loss.backward()
          self.optimizer.step()
          if change_lr:
            self.scheduler.step()
          _, predicts = torch.max(outputs, 1)
          train_error += (predicts != labels).sum().item()
          train_loss += loss
        train_loss /= train_batch_num
        train_error /= (self.batch_size * train_batch_num)
        train_error *= 100
        with torch.no_grad():
          #validate_error = 0
          #validate_batch_num = len(self.dataloaders['validate'])
          #for index, data in enumerate(self.dataloaders['validate']):
          #  inputs, labels = data
          #  inputs = inputs.to('cuda' if self.gpu else 'cpu')
          #  labels = labels.to('cuda' if self.gpu else 'cpu')
          #  outputs = self.model(inputs)
          #  _, predicts = torch.max(outputs, 1)
          #  validate_error += (predicts != labels).sum().item()
          #validate_error /= (self.batch_size * validate_batch_num)
          #validate_error *= 100
          self.model.eval()
          test_loss = 0
          test_error = 0
          test_batch_num = len(self.dataloaders['test'])
          for index, data in enumerate(self.dataloaders['test']):
            inputs, labels = data
            inputs = inputs.to('cuda' if self.gpu else 'cpu')
            labels = labels.to('cuda' if self.gpu else 'cpu')
            outputs = self.model(inputs)
            _, predicts = torch.max(outputs, 1)
            test_error += (predicts != labels).sum().item()
            loss = self.criterion(outputs, labels)
            test_loss += loss
          test_loss /= test_batch_num
          test_error /= (self.batch_size * test_batch_num)
          test_error *= 100
        print(f"Training Error: {train_error}")
        print(f"Testing Error: {test_error}")
        if train_error < best_train_err or True:
          best_train_err = train_error
          best_train_loss = train_loss
          best_test_err = test_error
          best_test_loss = test_loss
          #best_val_err = validate_error
        epoch_test_err.append(test_error)
        epoch_test_loss.append(float(test_loss.to('cpu')))
      return best_train_err, float(best_train_loss.to('cpu')), best_test_err, float(best_test_loss.to('cpu')), epoch_test_err, epoch_test_loss 

    def experiment_epoch_num(self, epochs):
      train_losses = []
      train_errors = []
      test_losses = []
      test_errors = []
      for epoch in range(epochs):
        print("Epoch " + str(epoch + 1) + ":")
        self.model.train()
        train_batch_num = len(self.dataloaders['train'])
        train_loss = 0
        train_error = 0
        for index, data in enumerate(self.dataloaders['train']):
          inputs, labels = data
          inputs = inputs.to('cuda' if self.gpu else 'cpu')
          labels = labels.to('cuda' if self.gpu else 'cpu')
          self.optimizer.zero_grad()
          outputs = self.model(inputs)
          loss = self.criterion(outputs, labels)
          loss.backward()
          self.optimizer.step()
          _, predicts = torch.max(outputs, 1)
          train_error += (predicts != labels).sum().item()
          train_loss += loss
        train_loss /= train_batch_num
        train_error /= (self.batch_size * train_batch_num)
        train_error *= 100
        train_losses.append(float(train_loss.to('cpu')))
        train_errors.append(train_error)
        print(f"Train Error: {train_error}%")
        self.model.eval()
        test_loss = 0
        test_error = 0
        test_batch_num = len(self.dataloaders['test'])
        with torch.no_grad():
          for index, data in enumerate(self.dataloaders['test']):
            inputs, labels = data
            inputs = inputs.to('cuda' if self.gpu else 'cpu')
            labels = labels.to('cuda' if self.gpu else 'cpu')
            outputs = self.model(inputs)
            _, predicts = torch.max(outputs, 1)
            test_error += (predicts != labels).sum().item()
            loss = self.criterion(outputs, labels)
            test_loss += loss
          test_loss /= test_batch_num
          test_error /= (self.batch_size * test_batch_num)
          test_error *= 100
          test_losses.append(float(test_loss.to('cpu')))
          test_errors.append(test_error)
          print(f"Test Error: {test_error}%")
      np.save(os.path.join(path, "alex_epoch_train.npy"), train_errors)
      np.save(os.path.join(path, "alex_epoch_train_loss.npy"), train_losses)
      np.save(os.path.join(path, "alex_epoch_test.npy"), test_errors)
      np.save(os.path.join(path, "alex_epoch_test_loss.npy"), test_losses)

In [ ]:
train_losses = []
train_errors = []
test_losses = []
test_errors = []
widths = [1, 2, 3, 4, 6, 10, 14, 16, 22, 30, 38, 46, 54, 62, 78, 94, 110, 126, 180, 308]
for width in widths:
  print("Width2 Size: " + str(width))
  cnn = Width_net2(width)
  classifier = Classifier(cnn, dataloaders, batch_size)
  tre, trl, te, tl, epoch_test_err, epoch_test_loss = classifier.experiment_filter_num(1000)
  print(f"Training Error: {tre}")
  print(f"Testing Error: {te}")
  print(f"Training Loss: {trl}")
  print(f"Testing Loss: {tl}")
  train_losses.append(trl)
  test_losses.append(tl)
  train_errors.append(tre)
  test_errors.append(te)
  np.save(os.path.join(path, "test_error_wdith2_" + str(width) + "_.npy"), epoch_test_err)
  np.save(os.path.join(path, "test_loss_wdith2_" + str(width) + "_.npy"), epoch_test_loss)
  np.save(os.path.join(path, "cnn_width2_train.npy"), train_errors)
  np.save(os.path.join(path, "cnn_width2_train_loss.npy"), train_losses)
  np.save(os.path.join(path, "cnn_width2_test.npy"), test_errors)
  np.save(os.path.join(path, "cnn_width2_test_loss.npy"), test_losses) 

In [ ]:
train_losses = []
train_errors = []
test_losses = []
test_errors = []
widths = [1, 2, 3, 4, 10, 14, 19, 22, 30, 38, 46, 62, 78, 94, 126, 180, 308]
for width in widths:
  print("Width1 Size: " + str(width))
  cnn = Width_net1(width)
  classifier = Classifier(cnn, dataloaders, batch_size)
  tre, trl, te, tl, epoch_test_err, epoch_test_loss = classifier.experiment_filter_num(1000)
  print(f"Training Error: {tre}")
  print(f"Testing Error: {te}")
  print(f"Training Loss: {trl}")
  print(f"Testing Loss: {tl}")
  train_losses.append(trl)
  test_losses.append(tl)
  train_errors.append(tre)
  test_errors.append(te)
  np.save(os.path.join(path, "test_error_wdith1_" + str(width) + "_.npy"), epoch_test_err)
  np.save(os.path.join(path, "test_loss_wdith1_" + str(width) + "_.npy"), epoch_test_loss)
  np.save(os.path.join(path, "cnn_width1_train.npy"), train_errors)
  np.save(os.path.join(path, "cnn_width1_train_loss.npy"), train_losses)
  np.save(os.path.join(path, "cnn_width1_test.npy"), test_errors)
  np.save(os.path.join(path, "cnn_width1_test_loss.npy"), test_losses) 

In [ ]:
import matplotlib.pyplot as plt
train_error = np.load(os.path.join(path, "cnn_width2_train.npy"), allow_pickle = True).tolist()
train_loss = np.load(os.path.join(path, "cnn_width2_train_loss.npy"), allow_pickle = True).tolist()
test_error = np.load(os.path.join(path, "cnn_width2_test.npy"), allow_pickle = True).tolist()
test_loss = np.load(os.path.join(path, "cnn_width2_test_loss.npy"), allow_pickle = True).tolist()
epochs = [i + 1 for i in range(len(train_error))]
plt.plot(epochs, train_error)
plt.plot(epochs, test_error)
plt.legend(["train", "test"])
plt.title("Training and Testing Errors")
plt.show()
plt.plot(epochs, train_loss)
plt.plot(epochs, test_loss)
plt.legend(["train", "test"])
plt.title("Training and Testing Losses")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
train_error = np.load(os.path.join(path, "cnn_width3_train.npy"), allow_pickle = True).tolist()
train_loss = np.load(os.path.join(path, "cnn_width3_train_loss.npy"), allow_pickle = True).tolist()
test_error = np.load(os.path.join(path, "cnn_width3_test.npy"), allow_pickle = True).tolist()
test_loss = np.load(os.path.join(path, "cnn_width3_test_loss.npy"), allow_pickle = True).tolist()
epochs = [i + 1 for i in range(len(train_error))]
plt.plot(epochs, train_error)
plt.plot(epochs, test_error)
plt.legend(["train", "test"])
plt.title("Training and Testing Errors")
plt.show()
plt.plot(epochs, train_loss)
plt.plot(epochs, test_loss)
plt.legend(["train", "test"])
plt.title("Training and Testing Losses")
plt.show()